In [24]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

## Example: automobile data


https://www.kaggle.com/toramky/automobile-dataset

In [62]:
carsdata=pd.read_csv('../data/Automobile_data.csv')
carsdata.sample(5)

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
87,1,125,mitsubishi,gas,turbo,four,sedan,fwd,front,96.3,...,110,spdi,3.17,3.46,7.5,116,5500,23,30,9279
136,3,150,saab,gas,turbo,two,hatchback,fwd,front,99.1,...,121,mpfi,3.54,3.07,9.0,160,5500,19,26,18150
56,3,150,mazda,gas,std,two,hatchback,rwd,front,95.3,...,70,4bbl,?,?,9.4,101,6000,17,23,11845
123,-1,74,plymouth,gas,std,four,wagon,fwd,front,103.3,...,122,2bbl,3.35,3.46,8.5,88,5000,24,30,8921
188,2,94,volkswagen,gas,std,four,sedan,fwd,front,97.3,...,109,mpfi,3.19,3.4,10.0,100,5500,26,32,9995


To fit a model to find relevant predictors of `price`, let's first experiment with a handful of arbitrary covariates to fit an initial model.

In [65]:
cars=carsdata[['engine-size','horsepower','city-mpg','price']].copy()
#names have been giving me troubles..
cars.columns = ['enginesize', 'horsepower', 'citympg', 'price']

In [67]:
#deal with missing values: the missing values here are "?"
cars['enginesize'].replace('?', np.nan, inplace= True)
cars['horsepower'].replace('?', np.nan, inplace= True)
cars['citympg'].replace('?', np.nan, inplace= True)
cars['price'].replace('?', np.nan, inplace= True)
cars.isnull().sum()

enginesize    0
horsepower    2
citympg       0
price         4
dtype: int64

In [69]:
#Not too many missing values, for now let's drop them
#Recall how we could impute missing values from EDA
carsnew=cars.dropna()
carsnew.info()
carsnew.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 199 entries, 0 to 204
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   enginesize  199 non-null    int64 
 1   horsepower  199 non-null    object
 2   citympg     199 non-null    int64 
 3   price       199 non-null    object
dtypes: int64(2), object(2)
memory usage: 7.8+ KB


enginesize    0
horsepower    0
citympg       0
price         0
dtype: int64

In [80]:
#one more problem: price is an object, not int64(numeric), so is horsepower!
carsnew.loc[:,'price']=carsnew['price'].astype('int64')
carsnew.loc[:,'horsepower']=carsnew['horsepower'].astype('int64')
carsnew.info()

<class 'pandas.core.frame.DataFrame'>
Index: 199 entries, 0 to 204
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   enginesize  199 non-null    int64
 1   horsepower  199 non-null    int64
 2   citympg     199 non-null    int64
 3   price       199 non-null    int64
dtypes: int64(4)
memory usage: 7.8 KB


In [82]:
carsnew.info()

<class 'pandas.core.frame.DataFrame'>
Index: 199 entries, 0 to 204
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   enginesize  199 non-null    int64
 1   horsepower  199 non-null    int64
 2   citympg     199 non-null    int64
 3   price       199 non-null    int64
dtypes: int64(4)
memory usage: 7.8 KB


### Now we can fit the MLR model: price ~ enginesize + citympg+ horsepower

In [85]:
reg = smf.ols('price ~ enginesize + citympg + horsepower', data = carsnew).fit()

#print(dir(reg)) #members of the object provided by the modelling

In [87]:
#Model summary
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.794
Model:                            OLS   Adj. R-squared:                  0.791
Method:                 Least Squares   F-statistic:                     251.2
Date:                Wed, 11 Sep 2024   Prob (F-statistic):           1.03e-66
Time:                        15:00:36   Log-Likelihood:                -1912.4
No. Observations:                 199   AIC:                             3833.
Df Residuals:                     195   BIC:                             3846.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -2547.0951   2914.668     -0.874      0.383   -8295.415    3201.225
enginesize   124.3388     10.951     11.355      0.000     102.742     145.935
citympg     -151.3184     70.849     -2.136      0.034    -291.047     -11.590
horsepower    37.0876     16.262      2.281      0.024       5.016      69.159
==============================================================================
Omnibus:                       10.990   Durbin-Watson:                   0.771
Prob(Omnibus):                  0.004   Jarque-Bera (JB):               17.130
Skew:                           0.312   Prob(JB):                     0.000191
Kurtosis:                       4.295   Cond. No.                     1.96e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.96e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

<b>ANOVA F-test</b>

- From global anova: the full model significantly reduces SSE compared to the null model [Prob (F-statistic):	1.03e-66]

- From t test: enginesize, citympg, horsepower are all significant predictors of price, if we only test for each one individually. 

But recall that when we extract `anova_lm` seperately, different value of `type` can give us the results of either <b>sequential F test</b> results or <b> partial F test </b> results. 

In [141]:
# typ=1 and typ=2 in anova_lm gives different values in SS and F test
sm.stats.anova_lm(reg, type=1)

,df,sum_sq,mean_sq,F,PR(>F)
enginesize,1.0,9.625888e+09,9.625888e+09,724.441229,1.387785e-67
citympg,1.0,3.186064e+08,3.186064e+08,23.978216,2.039654e-06
horsepower,1.0,6.911228e+07,6.911228e+07,5.201368,2.365037e-02
Residual,195.0,2.591029e+09,1.328733e+07,NaN,NaN


In [143]:
sm.stats.anova_lm(reg, type=2)

,df,sum_sq,mean_sq,F,PR(>F)
enginesize,1.0,9.625888e+09,9.625888e+09,724.441229,1.387785e-67
citympg,1.0,3.186064e+08,3.186064e+08,23.978216,2.039654e-06
horsepower,1.0,6.911228e+07,6.911228e+07,5.201368,2.365037e-02
Residual,195.0,2.591029e+09,1.328733e+07,NaN,NaN


In [146]:
# typ-3 is the same as type=2, just added a test on intercept
sm.stats.anova_lm(reg, type=3)

,df,sum_sq,mean_sq,F,PR(>F)
enginesize,1.0,9.625888e+09,9.625888e+09,724.441229,1.387785e-67
citympg,1.0,3.186064e+08,3.186064e+08,23.978216,2.039654e-06
horsepower,1.0,6.911228e+07,6.911228e+07,5.201368,2.365037e-02
Residual,195.0,2.591029e+09,1.328733e+07,NaN,NaN


In [96]:
#Take a closer look at type=1: reduced model without citympg
reg_reduced_1 = smf.ols('price~enginesize',data=carsnew).fit()
sm.stats.anova_lm(reg_reduced_1, type=1)


,df,sum_sq,mean_sq,F,PR(>F)
enginesize,1.0,9.625888e+09,9.625888e+09,636.609809,1.265067e-63
Residual,197.0,2.978748e+09,1.512055e+07,NaN,NaN


In [98]:
#Take a closer look at type=1: full model with citympg
reg_full_1 = smf.ols('price~enginesize+citympg',data=carsnew).fit()
sm.stats.anova_lm(reg_full_1, type=1)



,df,sum_sq,mean_sq,F,PR(>F)
enginesize,1.0,9.625888e+09,9.625888e+09,709.238315,4.852522e-67
citympg,1.0,3.186064e+08,3.186064e+08,23.475016,2.566218e-06
Residual,196.0,2.660141e+09,1.357215e+07,NaN,NaN


The difference in SSE is the same as the SS for citympg in y~enginesize+citympg+horsepower

In [101]:
sm.stats.anova_lm(reg, type=1)

,df,sum_sq,mean_sq,F,PR(>F)
enginesize,1.0,9.625888e+09,9.625888e+09,724.441229,1.387785e-67
citympg,1.0,3.186064e+08,3.186064e+08,23.978216,2.039654e-06
horsepower,1.0,6.911228e+07,6.911228e+07,5.201368,2.365037e-02
Residual,195.0,2.591029e+09,1.328733e+07,NaN,NaN


Change orders in typ=1 changes the result because you are comparing different reduced and full models now.

In [104]:
reg_order= smf.ols('price~citympg+horsepower+enginesize',data=carsnew).fit()
sm.stats.anova_lm(reg_order, type=1)

,df,sum_sq,mean_sq,F,PR(>F)
citympg,1.0,5.988084e+09,5.988084e+09,450.661312,1.369003e-52
horsepower,1.0,2.312434e+09,2.312434e+09,174.033076,8.065939e-29
enginesize,1.0,1.713088e+09,1.713088e+09,128.926426,2.904006e-23
Residual,195.0,2.591029e+09,1.328733e+07,NaN,NaN


<b> From sequential F tests </b>

- We can kind of "rank" the importance of the predictors. 
- Since in the above example all predictors are significant, we will leave this to the next example. 

In [107]:
#Take a closer look at typ=2: reduced model without citympg
reg_reduced_2 = smf.ols('price~enginesize+horsepower',data=carsnew).fit()
sm.stats.anova_lm(reg_reduced_2, type=2)
#the difference in SSE is the SS of citympg in the full model

,sum_sq,df,F,PR(>F)
enginesize,1.672205e+09,1.0,123.603577,1.403756e-22
horsepower,3.271074e+08,1.0,24.178641,1.852652e-06
Residual,2.651640e+09,196.0,NaN,NaN


In [109]:
sm.stats.anova_lm(reg, type=2)

,sum_sq,df,F,PR(>F)
enginesize,1.713088e+09,1.0,128.926426,2.904006e-23
citympg,6.061127e+07,1.0,4.561585,3.394397e-02
horsepower,6.911228e+07,1.0,5.201368,2.365037e-02
Residual,2.591029e+09,195.0,NaN,NaN


<b> From partial F tests </b>

- Every reduced model is compared to the same full model:

For example, the F test for `enginesize` is comparing 

- Reduced model: price ~ citympg+ horsepower
- Full model: price ~ enginesize+ citympg+ horsepower

So it will suggest the signifcance of one predictor given ALL OTHER predictors are already in the model; we can use it to 

- judge the "conditional" importance of each predictor
- pick some reduced models with one less preditor that's better than the full model. 

<b>Example: Credit data</b>

In [118]:
# example: Credit data
credit = pd.read_csv("../data/Credit.csv")
credit.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  400 non-null    int64  
 1   Income      400 non-null    float64
 2   Limit       400 non-null    int64  
 3   Rating      400 non-null    int64  
 4   Cards       400 non-null    int64  
 5   Age         400 non-null    int64  
 6   Education   400 non-null    int64  
 7   Gender      400 non-null    object 
 8   Student     400 non-null    object 
 9   Married     400 non-null    object 
 10  Ethnicity   400 non-null    object 
 11  Balance     400 non-null    int64  
dtypes: float64(1), int64(7), object(4)
memory usage: 37.6+ KB


In [120]:
credit['Income'] = pd.to_numeric(credit['Income'])

In [122]:
model =smf.ols('Balance~Income + Limit + Rating + Age',data=credit).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Balance   R-squared:                       0.877
Model:                            OLS   Adj. R-squared:                  0.876
Method:                 Least Squares   F-statistic:                     705.6
Date:                Wed, 11 Sep 2024   Prob (F-statistic):          2.16e-178
Time:                        15:00:48   Log-Likelihood:                -2599.9
No. Observations:                 400   AIC:                             5210.
Df Residuals:                     395   BIC:                             5230.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -445.1048     40.576    -10.970      0.000    -524.877    -365.332
Income        -7.6127      0.382    -19.945      0.000      -8.363      -6.862
Limit          0.0818      0.045      1.834      0.067      -0.006       0.170
Rating         2.7314      0.664      4.111      0.000       1.425       4.038
Age           -0.8561      0.478     -1.789      0.074      -1.797       0.084
==============================================================================
Omnibus:                       94.733   Durbin-Watson:                   1.906
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              165.919
Skew:                           1.374   Prob(JB):                     9.36e-37
Kurtosis:                       4.550   Cond. No.                     2.65e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.65e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

From individual t test, Income and Rating are significant, Limit is on the boundary.So t test suggests the model: y~Income+Rating

In [125]:
# try sequential anova with the same order

sm.stats.anova_lm(model, type=1)



,df,sum_sq,mean_sq,F,PR(>F)
Income,1.0,1.813117e+07,1.813117e+07,691.691426,7.902872e-89
Limit,1.0,5.533791e+07,5.533791e+07,2111.102870,1.465723e-160
Rating,1.0,4.328357e+05,4.328357e+05,16.512381,5.832806e-05
Age,1.0,8.394134e+04,8.394134e+04,3.202304,7.430058e-02
Residual,395.0,1.035406e+07,2.621280e+04,NaN,NaN


- p-values disagree with t tests
- suggest y\~Income+Limit is sig. better than just y\~Income
- y~ Income+Limit+ Rating is the best one (adding age is not significant)

In [128]:
#Put the questionable one (Limit) in the end 
model_2 =smf.ols('Balance~Income + Rating +Limit',data=credit).fit()
sm.stats.anova_lm(model_2, type=1)


,df,sum_sq,mean_sq,F,PR(>F)
Income,1.0,1.813117e+07,1.813117e+07,687.865947,1.316473e-88
Rating,1.0,5.567620e+07,5.567620e+07,2112.261356,8.086517e-161
Limit,1.0,9.454487e+04,9.454487e+04,3.586873,5.896627e-02
Residual,396.0,1.043800e+07,2.635858e+04,NaN,NaN


Now it suggest y~Income+Rating might be enough, adding Limit and Age doesn't significantly improve it anymore.

- limit might be on the boundary

<b>Can you try typ=2?</b> What does that tell us?

For the candidates, let's look at R^2 and adj-R^2

In [133]:
model_c1 = smf.ols('Balance~Income + Rating ',data=credit).fit() #from t test and anova(typ=2)
model_c2 = smf.ols('Balance~Income + Rating +Limit',data=credit).fit()#from anova(typ=1)
#model_c3 = smf.ols('Balance~Income + Rating +Age',data=credit).fit() #not that strong

In [135]:
print(model_c1.rsquared, model_c1.rsquared_adj)
print(model_c2.rsquared, model_c2.rsquared_adj)
#print(model_c3.rsquared, model_c3.rsquared_adj)


0.8751179476994355 0.8744888189724805
0.8762389456262863 0.8753013618810309


Out of possible candidates , based on adjusted R^2, model_c2 is the best choice;

the analysis during the way suggests the possible ranking of importance:
Rating ~ Income ~ Rating > Limit > Age 

### Remark 

You can already see that different methods lead to different conclusions. After we learn MLR diagnostics and more model selection methods, we need to argue for a metric/criterion which is most appropriate for the real life context of our problem, and then choose the model that performs best according to it. If it is still hard to decide after doing this, you can try evaluating prediction errors on a test set and then select the model with the best prediction performance. 